# Chapter 14 - A/B Testing and MySQL


Note: you can install the requirements file for this notebook if you want to install all the libraries at once. Open a new terminal window, navigate to the folder with the "requirements_jupyter.txt" and run the following command:

pip3 install -r requirements_jupyter.txt


In [1]:
# !pip3 install -r requirements_jupyter.txt

In [2]:
import mysql.connector
from mysql.connector import MySQLConnection, Error


## Inserting a new row

In [3]:
try:
    cnx = mysql.connector.connect(user='webuser', password='thesecret', database='ABTesting')
    cursor = cnx.cursor()
    query = "INSERT INTO ABTesting.tblFrontPageOptions (uuid, liked, pageid) VALUES (%s, %s,%s);"
    args = (999, 0, 3444)
    cursor.execute(query, args)  
    cursor.close()
    cnx.commit()
    cnx.close()
except mysql.connector.Error as err:
    print("Something went wrong: {}".format(err))
    

## Read data in a table

In [4]:
import datetime
import mysql.connector

try:
    cnx = mysql.connector.connect(user='webuser', password='thesecret', database='ABTesting')
    cursor = cnx.cursor()
    query = "SELECT * FROM  ABTesting.tblFrontPageOptions"
    cursor.execute(query)

    for (uuid, liked, pageid, time_stamp) in cursor:
      print("uuid: {} liked:{} pageid: {} on {:%m/%d/%Y %H:%M:%S}".format(
        uuid, liked, pageid, time_stamp))

    cursor.close()
    cnx.close()
except mysql.connector.Error as err:
    print("Something went wrong: {}".format(err))
    

uuid: 9999 liked:1 pageid: 2 on 06/28/2018 14:13:36
uuid: 999 liked:1 pageid: 3444 on 06/28/2018 14:38:50
uuid: 2 liked:1 pageid: 1 on 06/28/2018 14:38:58
uuid: 999 liked:0 pageid: 3444 on 07/05/2018 12:10:38


## Update existing row

In [5]:
try:
    cnx = mysql.connector.connect(user='webuser', password='thesecret', database='ABTesting')
    cursor = cnx.cursor()
    query = "UPDATE ABTesting.tblFrontPageOptions SET liked = %s WHERE uuid = %s;"
    args = (1, 999)
    cursor.execute(query, args)  
    cursor.close()
    cnx.commit()
    cnx.close()
except mysql.connector.Error as err:
    print("Something went wrong: {}".format(err))

   ## Creating a unique user ID

In [6]:
import uuid
str(uuid.uuid4())

'f965137d-8b03-4461-8696-46d8004746a3'

## Web appliction functions

Abstract account settings

In [7]:
mysql_account = 'webuser'
mysql_password = 'thesecret'


The InsertInitialVisit function creates an entry whenever a user visits the page. It assumes the user doesn't like the page so enters thumbs down for the randomly assinged pageid

In [8]:
def InsertInitialVisit(uuid, pageid):
    try:
        cnx = mysql.connector.connect(user=mysql_account, password=mysql_password, database='ABTesting')
        cursor = cnx.cursor()
        query = "INSERT INTO ABTesting.tblFrontPageOptions (uuid, liked, pageid) VALUES (%s,%s,%s);"
        args = (uuid, 0, pageid)
        cursor.execute(query, args)  
        cursor.close()
        cnx.commit()
        cnx.close()
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))


InsertInitialVisit(2, 1)
        

The UpdateVisitWithLike function responds to when a user clicked the thumbs up button and updates the previously entered column with a thumbs up. In order to handle multiple users we pass the user id (uuid) back and forth to update the correct table row.

In [9]:
def UpdateVisitWithLike(uuid):
    try:
        cnx = mysql.connector.connect(user=mysql_account, password=mysql_password, database='ABTesting')
        cursor = cnx.cursor()
        query = "UPDATE ABTesting.tblFrontPageOptions SET liked = %s WHERE uuid = %s;"
        args = (1, uuid)
        cursor.execute(query, args)  
        cursor.close()
        cnx.commit()
        cnx.close()
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))

UpdateVisitWithLike(2)

Finally we create a reporting function to report on the total votes and the up and down votes.

In [10]:
def GetVoteResults():
    results = ''
    total_votes = 0
    total_up_votes = 0
    total_up_votes_page_1 = 0
    total_up_votes_page_2 = 0
    try:
        cnx = mysql.connector.connect(user=mysql_account, password=mysql_password, database='ABTesting')
        cursor = cnx.cursor()
        query = "SELECT * FROM  ABTesting.tblFrontPageOptions"
        cursor.execute(query)

        for (uuid, liked, pageid, time_stamp) in cursor:
            total_votes += 1
            if liked==1 and pageid==1:
                total_up_votes_page_1 += 1
            if liked==1 and pageid==2:
                total_up_votes_page_2 += 1
            if liked == 1:
                total_up_votes += 1
            results += ("uuid: {} liked:{} pageid: {} on {:%m/%d/%Y %H:%M:%S}".format(uuid, liked, pageid, time_stamp)) + "<br />"
        cursor.close()
        cnx.close()
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))
    
    return (results, total_votes, total_up_votes, total_up_votes_page_1, total_up_votes_page_2)


In [11]:
results, total_votes, total_up_votes, total_up_votes_page_1, total_up_votes_page_2 = GetVoteResults()

In [12]:
print("total_up_votes_page_1:", total_up_votes_page_1)

total_up_votes_page_1: 2


In [13]:
print("total_up_votes_page_2:", total_up_votes_page_2)

total_up_votes_page_2: 1
